In [0]:
%sql
DROP TABLE IF EXISTS uc_athlete_data.silver.strava_activities;


In [0]:
#dbutils.fs.rm("abfss://silver@adlsathlete.dfs.core.windows.net/strava/activities/strava_activities/", recurse=True)
#dbutils.fs.rm("abfss://silver@adlsathlete.dfs.core.windows.net/", recurse=True)

In [0]:
# 📥 1. Leitura da tabela da Bronze
df_bronze = spark.table("uc_athlete_data.bronze.strava_activities").filter("athlete_id IS NOT NULL")

In [0]:
# Filtrar registros com athlete_id nulo
df_errors = spark.read.table("uc_athlete_data.bronze.strava_activities").filter("athlete_id IS NULL")

# Salvar em nova tabela de erros (sobrescrevendo ou adicionando, conforme o caso)
df_errors.write.format("delta") \
    .mode("append") \
    .saveAsTable("uc_athlete_data.bronze.strava_activities_errors")


In [0]:
display(df_bronze)

In [0]:
%sql
select * from uc_athlete_data.bronze.strava_activities_errors

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

# Definindo a window
window_spec = Window.partitionBy("athlete_id","id","start_date").orderBy(col("ingestion_timestamp").desc())

# Aplicando row_number
df_ranked = df_bronze.withColumn("row_num", row_number().over(window_spec))

# Mantendo apenas a 1ª linha de cada grupo
df_deduplicated = df_ranked.filter(col("row_num") == 1).drop("row_num")


In [0]:
# 🧼 2. Transformações e limpeza
from pyspark.sql.functions import col, from_unixtime, round, date_format
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import to_date, round, col,when,expr
from pyspark.sql.functions import col, round, to_timestamp, to_date, when, date_format
from pyspark.sql.functions import floor, round, format_string, col
from pyspark.sql.functions import input_file_name, regexp_extract

df_silver = (
    df_deduplicated
    .withColumn("start_date", to_timestamp("start_date", "yyyy-MM-dd'T'HH:mm:ss'Z'"))
    .withColumn("start_date", to_date("start_date"))
    .withColumn("distance_km", round(col("distance") / 1000, 2))
    .withColumn("average_speed_kmh", round(col("average_speed") * 3.6, 2))
    .withColumn("pace_min_km", when(col("type") == "Run", round((col("elapsed_time") / 60) / (col("distance") / 1000), 2)).otherwise(0))
    .withColumn("pace_min_km_moving_time", when(col("type") == "Run", round((col("moving_time") / 60) / (col("distance") / 1000), 2)).otherwise(0))
    .withColumn("tempo_real", expr("format_string('%02d:%02d:%02d', int(moving_time/3600), int((moving_time%3600)/60), int(moving_time%60))"))
    .withColumn(
        "pace_min_km_new",
        when(
            col("distance") > 0,
            round((col("moving_time") / 60) / (col("distance") / 1000), 3)
        ).otherwise(None)
    )
    .withColumn(
    "pace_strava",
    format_string(
        "%d:%02d",
        floor(col("pace_min_km_new")),
        round((col("pace_min_km_new") - floor(col("pace_min_km_new"))) * 60, 0).cast("int")
        )
    )
    .withColumn("dia_semana", date_format("start_date", 'E'))
    .drop("resource_state", "map", "gear_id", "external_id", "upload_id")
    #.withColumn("file_path", _metadata.file_path)
    .select(
        "achievement_count",
        "athlete_count",
        "average_cadence",
        "average_heartrate",
        "elev_high",
        "elev_low",
        "flagged",
        "has_heartrate",
        "id",
        "location_city",
        "location_country",
        "location_state",
        "manual",
        "max_heartrate",
        "max_speed",
        "moving_time",
        "name",
        "pr_count",
        "sport_type",
        "start_date",
        "start_date_local",
        "timezone",
        "total_elevation_gain",
        "type",
        "workout_type",
        "athlete_id",
        "ingestion_timestamp",
        "elapsed_time",
        "distance_km",
        "average_speed_kmh",
        "tempo_real",
        "pace_min_km_moving_time",
        "pace_min_km",
        "pace_min_km_new",
        "pace_strava",
        "dia_semana"
    )
)


''' df_silver = df_silver \
    .withColumnRenamed("distance_km", "distancia_km") \
    .withColumnRenamed("average_speed_kmh", "velocidade_media_kmh") \
    .withColumnRenamed("max_speed_kmh", "velocidade_maxima_kmh") \
    .withColumnRenamed("pace_min_km", "ritmo_min_por_km") \
    .withColumnRenamed("gain_per_km", "altimetria_por_km") \
    .withColumnRenamed("duration_diff", "tempo_parado_segundos") \
    .withColumnRenamed("duration_min", "duracao_minutos") \
    .withColumnRenamed("start_date", "data_inicio") \
    .withColumnRenamed("start_date_only", "data_inicio_dia") \
    .withColumnRenamed("upload_day_of_week", "dia_da_semana") \
    .withColumnRenamed("upload_hour", "hora_do_dia") \
    .withColumnRenamed("has_gps", "possui_gps") \
    .withColumnRenamed("is_morning", "foi_pela_manha") \
    .withColumnRenamed("start_lat", "latitude_inicio") \
    .withColumnRenamed("start_lng", "longitude_inicio") \
    .withColumnRenamed("end_lat", "latitude_fim") \
    .withColumnRenamed("end_lng", "longitude_fim")'''

display(df_silver)
# 📤 3. Escrita na Silver como Delta
output_path = "abfss://silver@adlsathlete.dfs.core.windows.net/strava/activities/strava_activities/"

df_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(output_path)


In [0]:
%sql
CREATE TABLE IF NOT EXISTS uc_athlete_data.silver.strava_activities
USING DELTA
LOCATION 'abfss://silver@adlsathlete.dfs.core.windows.net/strava/activities/strava_activities/';


In [0]:
%sql
select type, count(*) from uc_athlete_data.silver.strava_activities
group by 1

In [0]:
%sql
select * from uc_athlete_data.silver.strava_activities
where type = 'Run'
and  ( name in ('Prova 10KM em Rotterdam -Holanda 🚀','Fluido no flow! Gostosinho') 
or start_date = '2025-05-18')
order by start_date_local desc

In [0]:
%sql
select count(id), count(distinct id) from uc_athlete_data.silver.strava_activities